In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve,least_squares,root

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
p = RollRig.base_params(3)

In [3]:
sm = mf.SymbolicModel.from_file('RollRigModel.py')
sm.ExtForces = ef.CompositeForce([sm.ExtForces,ef.CustomForce(None)])
sm = sm.subs({p.alpha_r.name:0,p.alpha_1.name:sym.atan(sym.sin(p.Lambda)*sym.sin(p.q[1])),
             p.alpha_2.name:sym.atan(sym.sin(p.Lambda)*sym.sin(p.q[2]))})

In [4]:
p.c.value = 0.067
p.s.value = 1
p.sigma.value = 0.28
p.m_w.value = 0.22
p.m_f.value = 0.038
p.m_l.value = 0#0.0275
p.alpha_r.value = 0
p.c_d_max.value = 1
p.a_0.value = 2*np.pi
p.a_1.value = 2*np.pi
#p.tau.value = 0
p.alpha_r.value = 0
p.w_g.value = 0
p.beta.value = 0.0036
p.V.value = 22.5

#p.s.value = 0.72
#p.sigma.value = 0
#p.m_f.value = 0
#p.m_l.value = 0


p.Lambda.value = np.deg2rad(10)

In [5]:
nm = mf.NumericModel.from_SymbolicModel(p,sm)
ext_f = nm.ExtForces.force_funcs[0]

torque_period = 0.1
torque_max = p.beta.value*p.V.value**2

def torque(tup,x,t):
    if t<2:
        return 0
    elif t<torque_period/2+2:
        return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
    else:
        return torque_max
    
def custom_force(tup,x,t):
    return np.array([[torque(tup,x,t)],[0],[0]])

nm.ExtForces.force_funcs = [ext_f , custom_force]



In [6]:
def GenRunData(model,p,ic,end_time,xNames=None,additional_cols = {},sample_freq=100):
    
    if xNames==None:
        xNames = [f'x{i}' for i in range(p.qs*2)]
    
    # create objective function and solve
    def objectiveDeriv(t,y):
        return nm.deriv(t,y,p.GetNumericTuple(y,t))
    yData = solve_ivp(objectiveDeriv,(0,end_time),ic,max_step=1/sample_freq)
    
    t = np.linspace(0,end_time,(end_time*100)+1)
   
    #interpolate to reduce t and y points
    int_func = interp1d(yData.t,yData.y)  
    yi = int_func(t)
    
    #generate list
    tau = [torque((0,),[0]*p.qs*2,i) for i in t]
    q_data = [{names[i]:v for i,v in enumerate(row)}for row in yi.T]    
    params = {k.name:v for k,v in p.GetSubs(0,None).items()}


    data = [{'t':_t,'torque':tau[i],'Locked':True,**q_data[i],**params,**additional_cols} for i,_t in enumerate(t)]
    return data
    

In [7]:
names = ['Roll','Roll Rate','Left FWT Angle','Left FWT Velocity','Right FWT Angle','Right FWT Velocity']

res = []

for lam in np.deg2rad([10,20,30]):
    p.Lambda.value = lam
    for camber in np.deg2rad([0,-10,10]):
        p.alpha_c.value = camber

        for v in np.linspace(15,30,4):
            p.V.value = v
            ext_f = nm.ExtForces.force_funcs[0]

            torque_period = 0.1
            for tm in [3,2,1]:
                p.beta.value = 0.0036 * tm /3  * 0.7    
                torque_max = p.beta.value*p.V.value**2

                def torque(tup,x,t):
                    if t<2:
                        return 0
                    elif t<torque_period/2+2:
                        return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
                    else:
                        return torque_max

                def custom_force(tup,x,t):
                    return np.array([[torque(tup,x,t)],[0],[0]])

                nm.ExtForces.force_funcs = [ext_f , custom_force]

                res+=GenRunData(nm,p,[0]*p.qs*2,10,names,{'Mode':'Free'},100)

In [8]:
df = pd.DataFrame(res)
df.to_pickle('FreeData.pkl')

In [9]:
df['Roll Rate [Deg]']=np.rad2deg(df['Roll Rate'])
df['Left FWT Angle [Deg]']=np.rad2deg(df['Left FWT Angle'])
df[df['t']>4].groupby(['V','Lambda','alpha_c'])['Roll Rate [Deg]'].mean()

V     Lambda    alpha_c  
15.0  0.174533  -0.174533    161.243943
                 0.000000    156.473513
                 0.174533    150.234474
      0.349066  -0.174533    157.768844
                 0.000000    160.406728
                 0.174533    159.198511
      0.523599  -0.174533    161.593782
                 0.000000    168.473928
                 0.174533    159.271256
20.0  0.174533  -0.174533    189.910372
                 0.000000    208.870448
                 0.174533    208.541237
      0.349066  -0.174533    216.930937
                 0.000000    218.745569
                 0.174533    219.668352
      0.523599  -0.174533    225.240368
                 0.000000    213.014674
                 0.174533    226.249528
25.0  0.174533  -0.174533    259.542361
                 0.000000    265.262778
                 0.174533    236.648798
      0.349066  -0.174533    279.070677
                 0.000000    264.724926
                 0.174533    281.729961
      0.523599

In [10]:
df[(df['t']<=1.5) & (df['t']>=1) & (df['alpha_c']==0)].groupby(['Mode','V','Lambda'])['Left FWT Angle [Deg]'].mean()

Mode  V     Lambda  
Free  15.0  0.174533    37.886177
            0.349066    21.604401
            0.523599    15.500748
      20.0  0.174533    23.811844
            0.349066    12.590976
            0.523599     8.690438
      25.0  0.174533    15.683158
            0.349066     8.126624
            0.523599     5.576968
      30.0  0.174533    10.988996
            0.349066     5.646647
            0.523599     3.868601
Name: Left FWT Angle [Deg], dtype: float64

In [11]:
df

,t,torque,Locked,Roll,Roll Rate,Left FWT Angle,Left FWT Velocity,Right FWT Angle,Right FWT Velocity,c,...,w_g,Mode,qtilde0,qtilde1,qtilde2,qtilde3,qtilde4,qtilde5,Roll Rate [Deg],Left FWT Angle [Deg]
0,0.00,0.000,True,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.067,...,0,Free,0,0,0,0,0,0,0.000000e+00,0.000000
1,0.01,0.000,True,5.449322e-20,2.229111e-17,0.005685,1.027557,0.005685,1.027557,0.067,...,0,Free,0,0,0,0,0,0,1.277187e-15,0.325742
2,0.02,0.000,True,2.324771e-19,-3.576715e-18,0.020902,2.006372,0.020902,2.006372,0.067,...,0,Free,0,0,0,0,0,0,-2.049307e-16,1.197590
3,0.03,0.000,True,1.029318e-19,-3.773946e-17,0.045606,2.923175,0.045606,2.923175,0.067,...,0,Free,0,0,0,0,0,0,-2.162312e-15,2.613027
4,0.04,0.000,True,-1.801571e-19,-2.429247e-17,0.079122,3.767125,0.079122,3.767125,0.067,...,0,Free,0,0,0,0,0,0,-1.391856e-15,4.533384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108103,9.96,0.756,True,2.370211e+01,1.406146e+00,-0.406110,7.174697,-0.155496,-11.593322,0.067,...,0,Free,0,0,0,0,0,0,8.056624e+01,-23.268378
108104,9.97,0.756,True,2.371680e+01,1.559892e+00,-0.327066,8.570409,-0.285213,-14.153375,0.067,...,0,Free,0,0,0,0,0,0,8.937526e+01,-18.739500
108105,9.98,0.756,True,2.373382e+01,1.867868e+00,-0.236385,9.459238,-0.434238,-15.419928,0.067,...,0,Free,0,0,0,0,0,0,1.070209e+02,-13.543868
108106,9.99,0.756,True,2.375453e+01,2.288286e+00,-0.140460,9.576760,-0.588831,-15.258218,0.067,...,0,Free,0,0,0,0,0,0,1.311091e+02,-8.047737


In [12]:
df.to_csv(r'/Users/fintan/Git/FlexiFWT_DAQ/Analysis/RollingRigV3/ModelComparison/FreeData.csv', index = False)